In [1]:
import json
import os
import PIL
import PIL.Image
from IPython.display import display
from PIL import Image

import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt


# Import all the Keras machinery we need
from keras import applications
from keras import optimizers
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import metrics

Using TensorFlow backend.


In [2]:
# Prepare our training / validation / etc set
def renameData():
    filename = './dataset/majurca-ecoclassifier-assets.json'

    #Read JSON data into the datastore variable
    if filename:
        with open(filename, 'r') as f:
            list_info = json.load(f)

    output_dir = './dataset_renamed'
    print(len(list_info))
    #Create folder for the test and training split
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
        
        k=0
        j=0

        for dict in list_info:
            #print(k)
            if dict['thumbnail_320x200'] != None:
                cam = dict['path'].replace('192-168-0-31','1')
                cam = cam.replace('192-168-0-32','2')
                cam = cam.replace('acquisitions','')
                if dict['tag_slugs'] != []:
                    new_name = (cam[:-4]+'_'+dict['tag_slugs'][0]+'.png')

                else:
                    new_name = (cam[:-4]+'_'+'not-labeled'+'.png')

                #print(new_name)

                new_path = output_dir + new_name
                name = dict['thumbnail_320x200_path']
                path = ('./dataset/'+ name)
                #print(k)
                #print('cp '+ path + ' ' + new_path)
                os.system('cp '+ path + ' ' + new_path)
                k = k+1

                

            else:
                j=j+1
        print('Done') 
        print(k)
        
renameData()


2896
Done
2894


In [58]:
#Giving same label to image from same group
size = len(groups)
labels2 = labels.copy()
print(len(labels2))
#print(labels)
d = {'label': labels,
     'group' : groups}        
data_info = pd.DataFrame(d)
print(data_info)


k = 1
group_label = labels[0]
group_size = 1
while k<size:
    #print(k)
    if groups[k] == groups[k-1]:
        if labels2[k] not in ['not-labeled','inconnu']:
            print(k)
            print('gone')
            group_label = labels2[k]
        group_size = group_size + 1
    else:
        if group_size > 1:
            labels2[(k-group_size):k] = [group_label]*group_size
            group_label = labels2[k]
        group_size = 1
    k = k + 1

d2 = {'label': labels2}        
data_info2 = pd.DataFrame(d2)
print(data_info2)
print(len(labels))  

2894
                 label  group
0          not-labeled      1
1          not-labeled      1
2         pe-hd-opaque      1
3            pet-fonce      2
4            pet-fonce      2
5          not-labeled      2
6           pet-opaque      3
7           pet-opaque      3
8           pet-opaque      3
9          not-labeled      4
10          godet-vide      4
11          godet-vide      4
12                 ela      5
13                 ela      5
14         not-labeled      5
15    plusieurs-objets      6
16    plusieurs-objets      6
17    plusieurs-objets      6
18         not-labeled      7
19          godet-vide      7
20         not-labeled      7
21          godet-vide      8
22          godet-vide      8
23          godet-vide      8
24         not-labeled      9
25          godet-vide      9
26          godet-vide      9
27          godet-vide     10
28          godet-vide     10
29         not-labeled     10
...                ...    ...
2864      pe-hd-opaque   1406
2865 

In [50]:
    def groupData():
        
        #Getting all the files names
        files = os.listdir('./dataset_renamed')
        files.sort()
        size = len(files)
        
        #Building a list of all the times
        acq_times = [] 
        for name in files:
            match = re.search('\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', name)
            #print(match.group())
            date = dt.strptime(match.group(), '%Y-%m-%d-%H-%M-%S')
            date = pd.to_datetime(date)
            acq_times.append(date)

        #Building a list of the time deltas
        deltas = []
        deltas.append(acq_times[0])
        for k in range(1,size):
             deltas.append(acq_times[k]-acq_times[k-1])
        
        
        #Grouping images depending on their deltas
        groups = []
        new_files = files.copy()
        group_count = 1;
        groups.append(group_count)
        for k in range(1,size):
            if deltas[k] < pd.Timedelta(seconds=4):
                groups.append(groups[k-1])
                '''
                if 'not-labeled' in new_files[k]:
                    lb = re.search('_\D+\.', files[k-1])
                    new_files[k]=new_files[k].replace('_not-labeled.',lb.group())
                if 'not-labeled' in new_files[k-1]:
                    lb = re.search('_\D+\.', files[k])
                    new_files[k-1]=new_files[k-1].replace('_not-labeled.',lb.group())
                '''
            else:
                group_count = group_count + 1 
                groups.append(group_count)
        
        
        #Building a list of the labels
        labels = []
        new_files = files.copy()
        for k, file in enumerate(files):
            #print(file)
            lb = re.search('_\S+\.', file)
            new_files[k] = new_files[k].replace(lb.group()[:-1],'')
            labels.append(lb.group()[1:-1]) # to remove the _ and the .        
        #print(files)
        #print(len(labels))
        
        
        # Giving same label to images in same group
        k = 1
        group_label = labels[0]
        group_size = 1
        while k<size:
            #print(k)
            if groups[k] == groups[k-1]:
                if labels[k] not in ['not-labeled','inconnu']:
                    group_label = labels[k]
                group_size = group_size + 1
            else:
                if group_size > 1:
                    labels[(k-group_size):k] = [group_label]*group_size
                    group_label = labels[k]
                group_size = 1
            k = k + 1

        d = {'filename': files,
             'new_filename': new_files,
             'label': labels,
             'group' : groups,
             'delta': deltas,
             'timestamp': acq_times}        
        
        
        
        data_info = pd.DataFrame(d)
        output_dir  = './dataset_grouped/'
        
        
        # image filter
        with open('filter_images.txt') as f:
            unwanted = f.read().splitlines()
        print("{} images were removed".format(len(unwanted)))
        '''
        for file in unwanted
            data_info.loc[data_info['filename'] == file]
        '''
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)

            k=0
            j=0

            for index, row in data_info.iterrows():
                if row['filename'] in unwanted:
                    data_info.drop(index,inplace = True)
                    
                else : 
                    new_name = (row['new_filename'][:-4] + '_' + row['label'] + '_grp'+str(row['group'])+'.png')
                    data_info.loc[index,'new_filename'] = new_name
                    new_path = output_dir + new_name
                    #print(new_path)
                    path = ('./dataset_renamed/'+ row['filename'])

                    #print('cp '+ path + ' ' + new_path)
                    os.system('cp '+ path + ' ' + new_path)
                
        

        
        print(data_info)
        return data_info
        
data_info = groupData()
# Saving data_info in a csv file:
file_name = 'data_info.csv'
data_info.to_csv(file_name)

234 images were removed
                                               filename  \
0       2019-04-05-09-51-29-283786-CAM2_not-labeled.png   
1       2019-04-05-09-51-29-998464-CAM1_not-labeled.png   
3         2019-04-05-09-51-48-921709-CAM2_pet-fonce.png   
4         2019-04-05-09-51-49-616585-CAM1_pet-fonce.png   
6        2019-04-05-09-52-05-060212-CAM2_pet-opaque.png   
7        2019-04-05-09-52-05-729671-CAM1_pet-opaque.png   
9       2019-04-05-09-52-17-287957-CAM2_not-labeled.png   
10       2019-04-05-09-52-17-973687-CAM1_godet-vide.png   
12              2019-04-05-09-52-25-418216-CAM2_ela.png   
13              2019-04-05-09-52-26-135206-CAM1_ela.png   
15    2019-04-05-10-05-05-932856-CAM2_plusieurs-obje...   
16    2019-04-05-10-05-06-621960-CAM1_plusieurs-obje...   
18      2019-04-05-10-07-05-763551-CAM2_not-labeled.png   
19       2019-04-05-10-07-06-456481-CAM1_godet-vide.png   
21       2019-04-05-10-07-13-788783-CAM2_godet-vide.png   
22       2019-04-05-10-07-14-488

In [53]:
# Creating the pair_info.csv

top = []
side = []
lb = []
waiting = 0
data_info.index = range(len(data_info))
group_memory = -1;
for index, row in data_info.iterrows():
    if row['group'] == group_memory:
        if 'CAM1' in row['new_filename']:
            top.append(row['new_filename'])

        if 'CAM2' in row['new_filename']:
            side.append(row['new_filename'])
        waiting = 0

    else:
        group_memory = row['group']
        lb.append(row['label'])

        if 'CAM1' in row['filename']:
            if waiting == 1:
                top.append('None')
            if waiting == 2:
                side.append('None')
            top.append(row['new_filename'])
            waiting = 2
        if 'CAM2' in row['filename']:
            if waiting == 1:
                top.append('None')
            if waiting == 2:
                side.append('None')
            side.append(row['new_filename'])
            waiting = 1

            
print(len(lb))
if waiting == 1:
    top.append('None')
if waiting == 2:
    side.append('None')       

d3 = {'cam1':top,
      'cam2':side,
      'label': lb}
print(d3)
pair_info= pd.DataFrame(d3)

print(pair_info)

file_name = 'pair_info.csv'
pair_info.to_csv(file_name)

1432
['2019-04-05-09-51-29-998464-CAM1_pe-hd-opaque_grp1.png', '2019-04-05-09-51-49-616585-CAM1_pet-fonce_grp2.png', '2019-04-05-09-52-05-729671-CAM1_pet-opaque_grp3.png', '2019-04-05-09-52-17-973687-CAM1_godet-vide_grp4.png', '2019-04-05-09-52-26-135206-CAM1_ela_grp5.png', '2019-04-05-10-05-06-621960-CAM1_plusieurs-objets_grp6.png', '2019-04-05-10-07-06-456481-CAM1_godet-vide_grp7.png', '2019-04-05-10-07-14-488516-CAM1_godet-vide_grp8.png', '2019-04-05-10-07-40-645192-CAM1_godet-vide_grp9.png', '2019-04-05-10-08-13-195095-CAM1_godet-vide_grp10.png'] ['2019-04-05-09-51-29-283786-CAM2_pe-hd-opaque_grp1.png', '2019-04-05-09-51-48-921709-CAM2_pet-fonce_grp2.png', '2019-04-05-09-52-05-060212-CAM2_pet-opaque_grp3.png', '2019-04-05-09-52-17-287957-CAM2_godet-vide_grp4.png', '2019-04-05-09-52-25-418216-CAM2_ela_grp5.png', '2019-04-05-10-05-05-932856-CAM2_plusieurs-objets_grp6.png', '2019-04-05-10-07-05-763551-CAM2_godet-vide_grp7.png', '2019-04-05-10-07-13-788783-CAM2_godet-vide_grp8.png', '2

In [55]:
# Creating the cam1.csv and cam2.csv
cam1_only = []
lb1 = []
cam2_only = []
lb2 = []

for index, row in pair_info.iterrows():
    
    if row['cam1'] != 'None':
        cam1_only.append(row['cam1'])
        lb1.append(row['label'])
    if row['cam2'] != 'None':
        cam2_only.append(row['cam2'])
        lb2.append(row['label'])

d1 = {'cam1': cam1_only,
    'label': lb1}
cam1_info = pd.DataFrame(d1)
cam1_info.to_csv('cam1_info.csv')

d2 = {'cam2': cam2_only,
    'label': lb2}
cam2_info = pd.DataFrame(d2)
cam2_info.to_csv('cam2_info.csv')


In [58]:
# Creating the pair_only_info.csv

pair_only_info = pair_info.copy()

for index, row in pair_only_info.iterrows():
    
    if row['cam1'] == 'None' or row['cam2'] == 'None':
        pair_only_info.drop(index,inplace = True)

pair_only_info.index = range(len(pair_only_info))

pair_only_info.to_csv('pair_only_info.csv')